In [1]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

from fastdownload import download_url

from fastai.vision.all import *
import timm
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import pickle

## Testing

In [2]:
page_t = requests.get('https://boardgamegeek.com/image/2669798')

In [3]:
soup_t = bs4(page_t.content, 'html.parser')

In [4]:
def extract_img(image_number): 
    page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
    soup = bs4(page.content, 'html.parser')
    uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images'))))
    patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')
    image_link=patt.findall(uncleaned_image_link) #add an assertion that the list is nonempty
    
    uncleaned_game_name=(str(soup.find('link',rel='canonical')))
    patt2= re.compile(str(image_number)+r'/[^"]*')
    game_name=patt2.findall(uncleaned_game_name) #add an assertion that the list is nonempty
    if len(image_link)==0 or len(game_name)==0:
        return 'invalid', uncleaned_image_link +'\n' + uncleaned_game_name +'\n' +str(soup)
    return game_name[0][8:], image_link[0]  #,page, soup, uncleaned_image_link, patt,

In [9]:
def get_link(image_number):
    page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
    if page.status_code==404:
        game_name='404'
        image_link=None
    else:
        soup = bs4(page.content, 'html.parser')
        uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
        patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')  
        image_link=patt.findall(uncleaned_image_link) #removes everything except the link iteself

        uncleaned_game_name=(str(soup.find('link',rel='canonical'))) 
        patt2= re.compile(str(image_number)+r'/[^"]*')
        game_name=patt2.findall(uncleaned_game_name)

        if image_link==[] or game_name==[]:
            game_name=str(page.status_code)
            image_link= None
        else: 
            game_name=game_name[0][8:]
            image_link=image_link[0]

    return np.asarray([image_number,game_name,image_link])

In [5]:
page_t = requests.get('https://boardgamegeek.com/image/'+str(2669812))
soup_t = bs4(page_t.content, 'html.parser')
uncleaned_image_link_t=(str(soup_t.find('link',href=re.compile(r'geekdo-images'))))
patt_t= re.compile(r'https.*'+str(2669812)+r'[^"]*')
image_link_t=patt_t.findall(uncleaned_image_link_t)

In [6]:
page_t.status_code==404

False

In [10]:
extract_img(2669798)

('gloomhaven',
 'https://cf.geekdo-images.com/HNMHfQ49oxs8ig-IEmNQWQ__imagepage/img/nNo4l2NjXToU3DCoBnMIbDExJu0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669798.png')

In [11]:
extract_img(2669900)

('jmaydet',
 'https://cf.geekdo-images.com/0WHEngboBp96fNQmqVNjiQ__imagepage/img/95blS1S21dYclnS_bMi7ws_pRq8=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2669900.jpg')

## Link Collector

In [26]:
def get_links_simple(start,stop):
    linklist=[]
    errorlist=[]
    for image_number in range(start,stop): 
        if image_number%10==0 and image_number!=0:
                print(image_number)
        page = requests.get('https://boardgamegeek.com/image/'+str(image_number))
        if page.status_code==404:
            errorlist.append([image_number,404,None])#None for vectorization
        else:
            soup = bs4(page.content, 'html.parser')
            uncleaned_image_link=(str(soup.find('link',href=re.compile(r'geekdo-images')))) #isolates the full link element to the image
            patt= re.compile(r'https.*'+str(image_number)+r'[^"]*')  
            image_link=patt.findall(uncleaned_image_link) #removes everything except the link iteself

            uncleaned_game_name=(str(soup.find('link',rel='canonical'))) 
            patt2= re.compile(str(image_number)+r'/[^"]*')
            game_name=patt2.findall(uncleaned_game_name)
            
            if image_link==[] or game_name==[]:
                errorlist.append([image_number, page.status_code,None]) #None for vectorization
            else: 
                linklist.append([image_number,game_name[0][8:],image_link[0]])
            
    return [linklist, errorlist]

### collecting links

In [40]:
l1 = get_links(2669798,2669806)

2669800


In [6]:
#l1

In [42]:
l2 = get_links(2669806,2669816)

In [7]:
#l2

In [10]:
l1=get_links_simple(2437871,2438871)

2437900
2438000
2438100
2438200
2438300
2438400
2438500
2438600
2438700
2438800


In [18]:
with open('game_links01', 'wb') as fp:
    pickle.dump(l1, fp)

In [12]:
with open ('game_links01', 'rb') as fp:
    l1_loaded = pickle.load(fp)

In [18]:
with open('testing', 'ab') as fp:
    pickle.dump([1,2,3,4,5], fp)

In [22]:
with open ('testing', 'rb') as fp:
    test_loaded = pickle.load(fp)

In [20]:
test_loaded

[1, 2, 3, 4]

In [136]:
l_save=[l2[0]+l3[0]+l4[0],l2[1]+l3[1]+l4[1]]

In [137]:
with open('game_links02', 'wb') as fp:
    pickle.dump(l_save, fp)

In [138]:
with open ('game_links02', 'rb') as fp:
    l_loaded = pickle.load(fp)

This took ~30 seconds

In [58]:
60*2*100

12000

So I could expect to get 12000 links in an hour

### some inspection

In [155]:
gns=pd.DataFrame(l_loaded[0])

In [156]:
gns

,0,1,2
0,2438871,entranced,https://cf.geekdo-images.com/U70FOKd_6j717C06XruJrw__imagepage/img/xgs8ngN6RvZfvop3stwcNGFHNck=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438871.png
1,2438872,entranced,https://cf.geekdo-images.com/SukFOA6_sH7xcLo7NF1LzQ__imagepage/img/wMigVCnMrN97yWkaszA10G0H670=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438872.png
2,2438873,entranced,https://cf.geekdo-images.com/wwlkbuFMPeGEQkPauyp-aA__imagepage/img/du5CWGW3Uwq4OqjIhcMUwu0VFdk=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438873.png
3,2438874,entranced,https://cf.geekdo-images.com/7c_AiD--joSVIU9GyGgerA__imagepage/img/BToCrDQBpNJOL_YSKF3qcySFQ8I=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438874.png
4,2438875,entranced,https://cf.geekdo-images.com/SzbQ0ZTlWuQh-f0TgRbTkQ__imagepage/img/xaYouPhVk-0ySBzZOB9Oxj_Ogzw=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2438875.png
...,...,...,...
832,2439566,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/wXxQmQwPpqQzreIGQdLypA__imagepage/img/Q0F2ceuSgzZwXAXprs3nhBEz6T8=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439566.jpg
833,2439567,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/unUjmsWNqTUb9-2qfit-Jg__imagepage/img/en5R5Wt_GGGPLSPDakcmNOPuK8k=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439567.jpg
834,2439568,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/iR0lfPph2RrmbNthTFRrhg__imagepage/img/5lejpE-Uq75ZcKR0ItMFaatYmL0=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439568.jpg
835,2439569,murder-baldurs-gate-launch-weekend,https://cf.geekdo-images.com/PPsbTfuNUAmCfbGj9sSb8A__imagepage/img/4wnsg6oFFI-r3UhIceZ68Uoj250=/fit-in/900x600/filters:no_upscale():strip_icc()/pic2439569.jpg


In [157]:
gns.columns=['image_id','game_title','image_link']

In [160]:
gns['game_title'].value_counts()

andrei123                          51
entranced                          45
                                   39
fogus                              30
mars-attacks-miniatures-game       30
                                   ..
cleric                              1
assassin                            1
jester                              1
mice-and-mystics                    1
fief-france-1429-buildings-pack     1
Name: game_title, Length: 190, dtype: int64

We need an effective way to isolate games from users

## Starting another test to use the gallery

In [161]:
page_t = requests.get('https://boardgamegeek.com/boardgame/174430/gloomhaven/images?pageid=1')
soup_t = bs4(page_t.content, 'html.parser')

In [162]:
page_t = requests.get('https://boardgamegeek.com/image/2437871/gloomhaven')
soup_t = bs4(page_t.content, 'html.parser')

In [163]:
soup_t.find_all("gg-app")

[<gg-app>
 <header class="skeleton-header">
 <nav>
 <span class="skeleton-header-logo"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 <span class="skeleton-header-item"></span>
 </nav>
 </header>
 <main class="skeleton-main">
 <div class="loader">
 <div class="line"></div>
 <div class="line"></div>
 <div class="line"></div>
 <div class="line"></div>
 </div>
 </main>
 </gg-app>]

In [164]:
soup_t.find_all("script")

[<script>window.AdSlots = window.AdSlots || {
 	cmd: [],
 	disableScripts: ['gpt'],
 	renderOnFirstLoad: false,
 	divCheck: false
 };</script>,
 <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-104725-1"></script>,
 <script>
 	window.dataLayer = window.dataLayer || [];
 	function gtag(){dataLayer.push(arguments);}
 	gtag('js', new Date());
 	gtag('consent', 'default', {
 		'ad_storage': 'denied',
 		'analytics_storage': 'denied',
 		'personalization_storage': 'denied',
 		'wait_for_update': 3000
 	});
 	gtag('config', 'UA-104725-1', {
 		'cookie_domain': 'boardgamegeek.com',
 		'send_page_view': false
 	});
 </script>,
 <script async="async" src="https://www.googletagservices.com/tag/js/gpt.js"></script>,
 <script>
 	var googletag = googletag || {};
 	var grecaptcha = grecaptcha || {};
 	var google = google || {};
 	googletag.cmd = googletag.cmd || [];
 
 	
 
 	window.GEEK = {
 		geeksite: 'boardgame',
 		recaptcha_public: '6Ldyr6EaAAAAAE0F2hYgtHHqbF6nKPTENAwo6SyU',

# Independent approach: Download via google with specific queries limiting to bgg

For example, the following seem to offer decent results for a difficult game query.

intitle:"ark nova" site:boardgamegeek.com

"ark nova" site:boardgamegeek.com

ark-nova site:boardgamegeek.com

For the game "On Mars", the third option produced the best results. For "Beyond the sun", the second option seemed best.

In [165]:
path = Path('boardgame-pictures-google')

Uncleaned Searches is for a dictionary giving the correct board game names in the end

In [3]:
searches='Gloomhaven','Brass Birmingham','Pandemic Legacy Season 1','Ark Nova','Terraforming Mars','Gloomhaven Jaws of the Lion','Twilight Imperium Fourth Edition','Star Wars Rebellion','Gaia Project','War of the Ring Second Edition','Spirit Island','Through the Ages A New Story of Civilization','Great Western Trail','Twilight Struggle','Dune Imperium','Scythe','The Castles of Burgundy','Nemesis','7 Wonders Duel','Brass Lancashire','Concordia','A Feast for Odin','Terra Mystica','Wingspan','Arkham Horror The Card Game','Clank Legacy Acquisitions Incorporated','Eclipse Second Dawn for the Galaxy','Root','Everdell','Viticulture Essential Edition','Orleans','Lost Ruins of Arnak','Mage Knight Board Game','Food Chain Magnate','Barrage','Marvel Champions The Card Game','Puerto Rico','Too Many Bones','Caverna The Cave Farmers','Blood Rage','Pax Pamir Second Edition','Agricola','Underwater Cities','The Crew Mission Deep Sea','Anachrony','Maracaibo','Mansions of Madness Second Edition','Pandemic Legacy Season 2','The Crew The Quest for Planet Nine','On Mars','Tzolkin The Mayan Calendar','Power Grid','Clans of Caledonia','Crokinole','Le Havre','Star Wars Imperial Assault','Cascadia','Kingdom Death Monster','Pandemic Legacy Season 0','Mechs vs Minions','Lisboa','The Quacks of Quedlinburg','The Gallerist','Paladins of the West Kingdom','Azul','Android Netrunner','Eclipse','Aeons End','The 7th Continent','Through the Ages A Story of Civilization','Race for the Galaxy','Clank A DeckBuilding Adventure','Five Tribes','Fields of Arle','Teotihuacan City of Gods','Grand Austria Hotel','Agricola Revised Edition','Robinson Crusoe Adventures on the Cursed Island','Kanban EV','The Voyages of Marco Polo','7 Wonders','Lords of Waterdeep','Dominant Species','Sleeping Gods','Great Western Trail Second Edition','Tainted Grail The Fall of Avalon','Architects of the West Kingdom','El Grande','Keyflower','Caylus','Battlestar Galactica The Board Game','Beyond the Sun','Mombasa','Dominion Intrigue','Troyes','Raiders of the North Sea','The Lord of the Rings Journeys in MiddleEarth','Twilight Imperium Third Edition','Eldritch Horror','Lorenzo il Magnifico'

In [167]:
uncleaned_searches="Gloomhaven","Brass: Birmingham","Pandemic Legacy: Season 1","Ark Nova","Terraforming Mars","Gloomhaven: Jaws of the Lion","Twilight Imperium: Fourth Edition","Star Wars: Rebellion","Gaia Project","War of the Ring: Second Edition","Spirit Island","Through the Ages: A New Story of Civilization","Great Western Trail","Twilight Struggle","Dune: Imperium","Scythe","The Castles of Burgundy","Nemesis","7 Wonders Duel","Brass: Lancashire","Concordia","A Feast for Odin","Terra Mystica","Wingspan","Arkham Horror: The Card Game","Clank!: Legacy – Acquisitions Incorporated","Eclipse: Second Dawn for the Galaxy","Root","Everdell","Viticulture Essential Edition","Orléans","Lost Ruins of Arnak","Mage Knight Board Game","Food Chain Magnate","Barrage","Marvel Champions: The Card Game","Puerto Rico","Too Many Bones","Caverna: The Cave Farmers","Blood Rage","Pax Pamir: Second Edition","Agricola","Underwater Cities","The Crew: Mission Deep Sea","Anachrony","Maracaibo","Mansions of Madness: Second Edition","Pandemic Legacy: Season 2","The Crew: The Quest for Planet Nine","On Mars","Tzolk'in: The Mayan Calendar","Power Grid","Clans of Caledonia","Crokinole","Le Havre","Star Wars: Imperial Assault","Cascadia","Kingdom Death: Monster","Pandemic Legacy: Season 0","Mechs vs. Minions","Lisboa","The Quacks of Quedlinburg","The Gallerist","Paladins of the West Kingdom","Azul","Android: Netrunner","Eclipse","Aeon's End","The 7th Continent","Through the Ages: A Story of Civilization","Race for the Galaxy","Clank!: A Deck-Building Adventure","Five Tribes","Fields of Arle","Teotihuacan: City of Gods","Grand Austria Hotel","Agricola (Revised Edition)","Robinson Crusoe: Adventures on the Cursed Island","Kanban EV","The Voyages of Marco Polo","7 Wonders","Lords of Waterdeep","Dominant Species","Sleeping Gods","Great Western Trail (Second Edition)","Tainted Grail: The Fall of Avalon","Architects of the West Kingdom","El Grande","Keyflower","Caylus","Battlestar Galactica: The Board Game","Beyond the Sun","Mombasa","Dominion: Intrigue","Troyes","Raiders of the North Sea","The Lord of the Rings: Journeys in Middle-Earth","Twilight Imperium: Third Edition","Eldritch Horror","Lorenzo il Magnifico"

In [168]:
search_dict=dict(zip(searches,uncleaned_searches))
#search_dict

### Copied an attempt, didn't work

https://www.geeksforgeeks.org/how-to-download-google-images-using-python/?tab=article

### Another attempt using Selenium
Generated basic script in Selenium IDE, then iterated upon it

In [ ]:
# Generated by Selenium IDE
import pytest
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55a783741463 <unknown>
#1 0x55a7835058d8 <unknown>
#2 0x55a78352b04a <unknown>
#3 0x55a783528c05 <unknown>
#4 0x55a78356c802 <unknown>
#5 0x55a78356c2af <unknown>
#6 0x55a783564443 <unknown>
#7 0x55a7835353c5 <unknown>
#8 0x55a783536531 <unknown>
#9 0x55a783793dce <unknown>
#10 0x55a783797192 <unknown>
#11 0x55a78377893e <unknown>
#12 0x55a783798103 <unknown>
#13 0x55a78376bd85 <unknown>
#14 0x55a7837b90a8 <unknown>
#15 0x55a7837b9239 <unknown>
#16 0x55a7837d4492 <unknown>
#17 0x7fbdcab64609 start_thread


In [10]:
browser = webdriver.Chrome("/home/jonas/programs/chromedriver")
browser.get('http://selenium.dev/')

/tmp/ipykernel_1460/1956462404.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/home/jonas/programs/chromedriver")


WebDriverException: Message: Service /home/jonas/programs/chromedriver unexpectedly exited. Status code was: 127


In [8]:
browser = webdriver.Chrome('../../programs/chromedriver')
browser.get('http://selenium.dev/')

/tmp/ipykernel_1460/1543986041.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('../../programs/chromedriver')


AttributeError: 'Service' object has no attribute 'process'

In [ ]:

class TestGloomhaven():
  def setup_method(self, method):
    self.driver = webdriver.Chrome()
    self.vars = {}
    self.link_list = []

  def teardown_method(self, method):
    self.driver.quit()
  
  def test_gloomhaven(self):
    # Test name: Gloomhaven
    # Step # | name | target | value
    # 1 | open | https://boardgamegeek.com/image/2437871/gloomhaven | 
    self.driver.get("https://boardgamegeek.com/image/2437871/gloomhaven")
    # 2 | click | css=.tw-right-0 > .svg-inline--fa > path | 
    #self.driver.find_element(By.CSS_SELECTOR, ".tw-right-0 > .svg-inline--fa > path").click()
    # 3 | executeScript | return window.location.href | pageurl
    #self.vars["pageurl"] = self.driver.execute_script("return window.location.href")
    for j in range(10):
        self.driver.find_element(By.CSS_SELECTOR, ".tw-right-0 > .svg-inline--fa > path").click()
        self.link_list.append(self.driver.execute_script("return window.location.href"))
    return self.link_list
    
  


In [19]:
tester=TestGloomhaven()

In [20]:
tester.setup_method()

TypeError: TestGloomhaven.setup_method() missing 1 required positional argument: 'method'

In [21]:
tester.test_gloomhaven()

AttributeError: 'TestGloomhaven' object has no attribute 'driver'

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home
